In [9]:
import matplotlib.pyplot as plt
import pickle

perturbation="0.1"

with open(f"results/sg_0.01_pert_{perturbation}_nruns_10.pkl", "rb") as f:
    results = pickle.load(f)


In [10]:
key = ("DDM", "{'drift_threshold': 5, 'warm_start': 5000}")
y_true, y_pred = results[key]["y_true"], results[key]["y_pred"]

In [11]:
from sklearn.metrics import accuracy_score, f1_score

print(accuracy_score(y_true, y_pred))
print(f1_score(y_true, y_pred))

0.55
0.18181818181818182
